In [1]:
# libraries
import pandas as pd
import os
import sys

# modules
sys.path.append(os.path.abspath("modules"))
import modules.dictionary as dict
from modules.classify import classify_keywords

# set your path here
os.chdir("input_data")
print(f'current directory: {os.getcwd()}')

current directory: /Users/tomokitakata/Desktop/research/dev/muldistribution/input_data


In [2]:
# target word(s) : class(es) assigned

override_map_2010 = {
    "気管食道科": ["気管食道内科", "気管食道外科"],
    "リ": "リウマチ科",  # 市立札幌病院
    ("神経科", "脳神"): ["脳神経内科" ,'脳神経外科'],
    "胃腸科": ["消化器内科", "消化器外科"],
    ("心臓血", "心臓", "心臓血管"): ["心臓・血管内科", '心臓・血管外科'],
    ("消化器科", "消化器" ,"消"): ["消化器内科", "消化器外科"],
    "循環器科": ['循環器内科', "循環器外科"],
    '呼吸器科': ['呼吸器内科', "呼吸器外科"],
    "内科・小児科・整形外科・外科・歯科・リハビリテーション科": ["一般内科", "小児科", "整形外科", "一般外科", "歯科・口腔顎顔面外科", "リハビリテーション科"],
    ("こう門科", "肛門科"): ["大腸・肛門内科", "大腸・肛門外科"],
    ("泌", "泌尿器科", "頻尿器下"): ["泌尿器内科", "泌尿器外科"],
    ("耳鼻いんこう科", "耳鼻いんこう", "耳鼻咽喉科", "耳鼻咽喉"): ["耳鼻咽喉・頭頸部内科", "耳鼻咽喉・頭頸部外科"],
    ("皮膚泌尿器科"): ["泌尿器内科", "泌尿器外科", "皮膚科"],
    }

In [3]:
# 分類対象となるキーワードの例
uniq_dpt = pd.read_csv('2010.csv', header=None).iloc[:,1]
uniq_dpt = uniq_dpt.to_list()
print(uniq_dpt)

['内科', '精神科', '神経科', '呼吸器科', '消化器科', '循環器科', '眼科', '耳鼻いんこう科', 'リハビリテーション科', '放射線科', '神経内科', '歯科', '外科', '脳神経外科', '皮膚科', '泌尿器科', '麻酔科', '整形外科', '心臓血管外科', '小児科', '産婦人科', '歯科口腔外科', '心臓血', '形成外科', '歯科口', 'リウマチ科', '呼吸器外科', '胃腸科', 'こう門科', '小児外科', '婦人科', '気管食道科', '心臓', 'リ', '心療内科', '小児歯科', '矯正歯科', 'アレルギー科', '産科', 'リハビリテ', '放射', '心臓血管', '脳神', '麻酔', 'リハビリ', '美容外科', '放射線', '歯', '耳鼻いんこう', '皮膚泌尿器科', '泌', 'リハビリテー', 'リハ', '麻', '消化器', 'リハビリテーショ', '消', 'リハビリテーシ', '性病科', '耳鼻咽喉科', '肛門科', '脳血管内科', '外来通院リハビリテーション', '循環器内科', '呼吸器内科', '肝臓内科', '人工透析内科', '消化器・血液内科', '頻尿器下', '耳鼻咽喉', '内科・小児科・整形外科・外科・歯科・リハビリテーション科']


In [4]:
classified_keywords = classify_keywords(
    keywords=uniq_dpt,               # List of unique medical department keywords to be classified
    classification_dict=dict.classification_dict,  # Dictionary defining classification rules for keywords
    multiple=True,                   # If True, allows matching with multiple classifications
    override_map=override_map_2010,   # Dictionary containing individual classification rules
    override_match_type="exact"       # Matching type for override_map classification ("exact" means exact match)
)

# Display the results in a DataFrame
max_length = max(len(values) for values in classified_keywords.values())
data = {
    cat: values + [""] * (max_length - len(values))
    for cat, values in classified_keywords.items()
}
result_df = pd.DataFrame(data)
result_df

,内科系,一般内科,呼吸器内科,耳鼻咽喉・頭頸部内科,気管食道内科,消化器内科,大腸・肛門内科,循環器内科,脳神経内科,脳血管内科,...,老年科,産婦人科,精神科,放射線科,救急・集中治療科,臨床検査科,病理診断科,緩和ケア科,鍼灸院,分類不可
0,内科,内科,呼吸器科,耳鼻いんこう科,気管食道科,消化器科,こう門科,循環器科,神経科,脳血管内科,...,,産婦人科,精神科,放射線科,,,,,,
1,神経内科,内科・小児科・整形外科・外科・歯科・リハビリテーション科,呼吸器内科,耳鼻いんこう,,胃腸科,肛門科,循環器内科,神経内科,,...,,婦人科,,放射,,,,,,
2,心療内科,,,耳鼻咽喉科,,消化器,,,脳神,,...,,産科,,放射線,,,,,,
3,脳血管内科,,,耳鼻咽喉,,消,,,,,...,,,,,,,,,,
4,循環器内科,,,,,消化器・血液内科,,,,,...,,,,,,,,,,
5,呼吸器内科,,,,,,,,,,...,,,,,,,,,,
6,肝臓内科,,,,,,,,,,...,,,,,,,,,,
7,人工透析内科,,,,,,,,,,...,,,,,,,,,,
8,消化器・血液内科,,,,,,,,,,...,,,,,,,,,,


In [5]:
result_df.columns

Index(['内科系', '一般内科', '呼吸器内科', '耳鼻咽喉・頭頸部内科', '気管食道内科', '消化器内科', '大腸・肛門内科',
       '循環器内科', '脳神経内科', '脳血管内科', '心臓・血管内科', '泌尿器内科', '肝・胆・膵内科', '腎臓内科',
       '内分泌内科', '糖尿病科', '代謝内科', '血液内科', '腫瘍内科', '感染症内科', '心療内科', '外科系', '一般外科',
       '呼吸器外科', '耳鼻咽喉・頭頸部外科', '気管食道外科', '消化器外科', '大腸・肛門外科', '循環器外科', '脳神経外科',
       '脳血管外科', '心臓・血管外科', '人工臓器・移植外科', '泌尿器外科', '肝・胆・膵外科', '腎臓外科', '内分泌外科',
       '腫瘍外科', 'アレルギー科', 'リウマチ科', '整形外科', '形成外科・美容外科', '皮膚科', '麻酔科',
       'リハビリテーション科', '眼科', '歯科・口腔顎顔面外科', '小児歯科', '小児科', '小児外科', '老年科', '産婦人科',
       '精神科', '放射線科', '救急・集中治療科', '臨床検査科', '病理診断科', '緩和ケア科', '鍼灸院', '分類不可'],
      dtype='object')

In [ ]:
result_df.to_csv('../classification_results/2010_classified.csv', encoding='utf-8', index=True)

In [7]:
result_df.to_excel('../classification_results/2010_classified.xlsx', index=True)